In [81]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM,Dense,Input,ConvLSTM2D,Reshape,Activation,Lambda,Softmax,Concatenate
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout,RepeatVector,Reshape,Activation,Lambda,Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import re
import pandas as pd
from keras import backend as K
import keras.backend as kerback
#from keras.models import Model
from tensorflow.keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow 

In [82]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy import random
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense

In [83]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [84]:
list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')

In [132]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer, vocab_size

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
        #print("max",max_length)
        #print("seq=",sequences)
        #print(vocab_size)
    return max_length

sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
print(sequences[0])
print(len(sequences[0])) #nombre d'element de tweet zero
nb_tweet=len(sequences)
batch_size=22
#print(len(tokenizer.word_index))
max_length=max_tweet(sequences)#nombre d'element de tout les tweet pour les mettres egaux
print(max_length)#############
print(vocab_size)

[146, 7, 46, 5, 2, 147, 148, 106, 3, 17, 149, 107, 1, 348, 349, 30, 350, 58, 351, 352, 3, 353, 354, 208, 52, 355, 356, 357, 7, 358, 3, 209, 47, 18, 359, 7, 360, 22, 4, 27]
40
57
841


In [180]:
def build_generator(n_outputs=57):
    model = Sequential()
    
    tweet=Input(shape=(22,n_outputs,))
    z0=Embedding(vocab_size+1,50, input_length=max_length)(tweet)
    z1=LSTM(units=1000)(z0)
    #z1=Reshape(( max_length, 50))(z0)
    print(z0.shape)
    noise=Input(shape=(22,n_outputs, 1))
    out=Embedding(vocab_size+1,50, input_length=max_length)(noise)
    out1=LSTM(units=1000)(out)
    #out2=Dense(units=50)(out1)
    #out2=Reshape(( max_length, 50) )(out2)
    print(out2)
    OO1=Concatenate(axis=2)([z1,out2])
    print(OO1.shape)
    #x0=Reshape((-1,max_length,50,2))(OO1)

    
    x1=LSTM(15, activation='relu', input_shape=(n_outputs,1),batch_size=20)(OO1)
    #15 nombre de neurone ,batch_size=n_batch=20 
    # repeat vector
    x2=RepeatVector(n_outputs)(x1)
    # decoder layer
    x3=LSTM(15, activation='relu', return_sequences=True)(x2)
    x4=TimeDistributed(Dense(1))(x3)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x4)
    dense_outputs = [Dense(57)(x) for x in unstacked]

    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)

    model = Model([noise,tweet], merged)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    print('generator')
    print(model.summary())
    return model
build_generator(n_outputs=57)

ValueError: Input 0 of layer lstm_223 is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: [None, 22, 57, 50]

In [165]:
def build_discriminator(n_outputs):
    model = Sequential()
    #x0=Input(shape=(tweet_size, 1))
    #x1=Flatten()(x0)
    tweet=Input(shape=(n_outputs,))
    z0=Embedding(vocab_size+1,50, input_length=max_length)(tweet)
    print(z0.shape)
    #z1=Reshape(( max_length, 50))(z0)
    
    noise=Input(shape=(n_outputs, 1))
    out1=LSTM(units=1000, return_state=False,return_sequences=True)(noise)
    out2=Dense(units=50)(out1)
    #out2=Reshape(( max_length, 50) )(out2)
    
    OO1=Concatenate(axis=2)([out2,z0])
    print(OO1.shape)
    
    x2=LSTM(15, activation='relu', input_shape=(n_outputs, 1))(OO1)
    x3=RepeatVector(n_outputs)(x2)
    x4=LSTM(15, activation='relu', return_sequences=True)(x3)
    x5=TimeDistributed(Dense(1))(x4)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x5)
    dense_outputs = [Dense(n_outputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)
    x1=Flatten()(merged)
    x6=Dense(1, activation='sigmoid')(x1)
    model = Model([noise,tweet], x6)
    model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])
    print('discriminator')
    print(model.summary())
    return model
build_discriminator(57)

(None, 57, 50)
(None, 57, 100)
discriminator
Model: "functional_129"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_196 (InputLayer)          [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
lstm_206 (LSTM)                 (None, 57, 1000)     4008000     input_196[0][0]                  
__________________________________________________________________________________________________
input_195 (InputLayer)          [(None, 57)]         0                                            
__________________________________________________________________________________________________
dense_233 (Dense)               (None, 57, 50)       50050       lstm_206[0][0]                   
________________________________________

In [166]:
# define the combined generator and discriminator model
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    
    inputs = Input(shape=(57,1))
    text_input = Input(shape=(57,1))
    
    text=generator([inputs,text_input])
  
    fake_pred = discriminator([text,inputs])
    
    gan = Model([inputs,text_input], fake_pred, name="gan")
    # compile model
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    print(gan.summary())
define_gan(build_generator(57), build_discriminator(57))

(None, 57, 100)
generator
Model: "functional_131"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_198 (InputLayer)          [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
input_197 (InputLayer)          [(None, 57)]         0                                            
__________________________________________________________________________________________________
lstm_209 (LSTM)                 (None, 57, 1000)     4008000     input_198[0][0]                  
__________________________________________________________________________________________________
embedding_73 (Embedding)        (None, 57, 50)       42100       input_197[0][0]                  
___________________________________________________________

In [167]:
def generate_real_samples(max_length):
    list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')
    sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
    #print(sequences[0])
    #print("len=",len(sequences[0]))
    X=sequences[:33]
    X=pad_sequences(X, maxlen=max_length, padding='post')
    y=sequences[33:]
    y=pad_sequences(y, maxlen=max_length, padding='post')
    X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.33)
    max_length=max_tweet(sequences)
    #t =sequences[:batch_size]#on a prix les 30 1er tweets
    #t = pad_sequences(t, maxlen=max_length, padding='post')
    #labels=sequences[:batch_size]
    #y = ones((n_samples, 1))
    #print("t:",t.shape)
    y = np.ones((batch_size, 1))
    return [X_train,Y_train],y

In [168]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim=57, n_samples=22, n_classes=vocab_size):
	# generate points in the latent space
	x_input = random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = random.randint(0, n_classes, n_samples)
	return [z_input, labels]

In [169]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim=57, n_samples=22):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    print(z_input.shape)
    print(labels_input.shape)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

In [170]:
# Create batch labels to use when calling train_on_batch
#y_real = ones((batch_size,1))
#y_fake = zeros((batch_size,1))

In [171]:
generator = build_generator(max_length)
# Build and compile the discriminator
discriminator = build_discriminator(max_length)
#discriminator.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])

# Build and compile the combined model
gan_model = define_gan(generator, discriminator)

(None, 57, 100)
generator
Model: "functional_135"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_204 (InputLayer)          [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
input_203 (InputLayer)          [(None, 57)]         0                                            
__________________________________________________________________________________________________
lstm_215 (LSTM)                 (None, 57, 1000)     4008000     input_204[0][0]                  
__________________________________________________________________________________________________
embedding_75 (Embedding)        (None, 57, 50)       42100       input_203[0][0]                  
___________________________________________________________

In [172]:
def sample_tweet(epoch,noise,g_model):
    result = g_model.predict(noise)
    #print(result)
    #print(result.shape)
    #print(len(result))
    res=[]
    predicted_word =""
    for j in range(57):
        #predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == int(result[0][j]):
                predicted_word = predicted_word+' '+word
                break
    res.append(predicted_word)
    print(res)
    
    #for i in res:
    #    print(i)
    return res

In [173]:
def disc_text(model_disc, seed_text):
    #print(seed_text)
    sequences, tokenizer=tokenizer_data(seed_text)
    sequences= pad_sequences(sequences, maxlen=max_length, padding='post')
    y = model_disc.predict(sequences)
    #print(y)
    #for i in y:
    if(i<0.7):
        print("fake")
    else:
        print("real")

In [175]:
# Store the losses
d_losses = []
g_losses = []
sample_period = 100 # every `sample_period` steps generate and save some data"
Epoch=1000
#x_real= generate_real_samples()
######################## Main training loop###########################
for i in range(Epoch):
        ########################
        ###Train Discriminator##
        #######################
        # prepare real samples
        #x_real= generate_real_samples()
        #print(x_real.shape)
        #print(x_real[-1])
        #noise_real=random.randn(1,57)
        #print(x_real.shape)
        #print(y_real.shape)
        #d_loss_real, d_acc_real=discriminator.train_on_batch(x_real, y_real)
        # Generate fake tweets
        #noise=random.randn(57,1)
        #x_fake= generator.predict([noise,x_real[-1]])
        #d_loss_fake, d_acc_fake =discriminator.train_on_batch([noise,x_fake[-1]], y_fake)
        #print("x_fake shape",x_fake.shape)
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(57)
        # update discriminator model weights
        d_loss1, d_acc_real = discriminator.train_on_batch([X_real, labels_real], y_real)
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(generator, 57, 22)
        # update discriminator model weights
        d_loss2, d_acc_fake = discriminator.train_on_batch([X_fake, labels], y_fake)
        # prepare points in latent space as input for the generator
        [z_input, labels_input] = generate_latent_points(57, 22)
        # create inverted labels for the fake samples
        y_gan = ones((33, 1))
        # update the generator via the discriminator's error
        g_loss = gan.train_on_batch([z_input, labels_input], y_gan)
        # Train the discriminator
        # both loss and accuracy are returned
        
        d_loss = 0.5 * (d_loss1 + d_loss2)
        d_acc  = 0.5 * (d_acc_real + d_acc_fake)
        
        #######################
        ### Train generator ###
        #######################
    
        #y_gan = ones((57, 1))
        #print(x_real[-1].shape)
        # update the generator via the discriminator's error
        #noise=random.randn(57,1)
        #gan_model([noise,x_real[-1]], y_gan).summary()
        #g_loss, n=gan_model.train_on_batch([noise,x_real[-1]], y_gan)
        
        # Save the losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        # evaluate discriminator on real examples
        #acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
        # evaluate discriminator on fake examples
        #acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
        
        #if i % 2 == 100:#after 100 epoch we save
        #if i % sample_period == 0:
        #print(f"epoch: {i+1}, d_loss: {d_loss:.2f}, \
        #d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
        #noiseg=random.randn(1,57)
        #sample_tweet(i,noiseg, generator)
        #disc_text(discriminator, tex)

(22, 57)
(22,)


ValueError: in user code:

    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\LENOVO\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\LENOVO